In [2]:
import dspy
import pandas as pd
from pyprojroot import here
import uuid

In [3]:
turbo = dspy.OpenAI(model='gpt-3.5-turbo')
dspy.settings.configure(lm=turbo)

In [4]:
class Valence(dspy.Signature):
    """Classify the following sentences as one of the following:
1. winning, including mentions of scoring points, victory, success, goals, solutions, best strategies
2. neutral information
3. losing, including information about death, losing points, lowering scores, forfeiting, losing lives, getting stuck or trapped

These sentences all came from an experiment in which participants played a game with squares of different colors moving on a board then wrote about it.
"""

    sentence = dspy.InputField(desc="A sentence to be tagged")
    tag = dspy.OutputField(desc="'abstract' if the message contains abstract or analogical verbs and 'concrete' otherwise")

In [5]:
class TagValence(dspy.Module):
    """A module to tag verbs for abstraction"""
    def __init__(self):
        super().__init__()
        
        self.tagger = dspy.Predict(Valence)

    def forward(self, sentence):
        return self.tagger(sentence=sentence)

class TagValenceWithCoT(dspy.Module):
    """A module to tag verbs for abstraction using chain of thought."""
    def __init__(self):
        super().__init__()
        
        self.tagger = dspy.ChainOfThought(Valence)

    def forward(self, sentence):
        return self.tagger(sentence=sentence)

In [6]:
df_tags1 = pd.read_csv(here("data/raw-data/ben_manually_tagged_sentences.csv"))
df_tags2 = pd.read_csv(here("data/raw-data/ben_manually_tagged_sentences_v2.csv"))
df_tags = pd.concat((df_tags1, df_tags2)).drop(columns="Unnamed: 0")
df_abstract_verbs = df_tags[df_tags["tags"].apply(lambda x: "abstract_verbs" in x)]